In [1]:
%pip install --use-feature=2020-resolver auto-sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/9b/f9/55/8c41e64eaeca2f2bdee8c17cfc44da812fe8fa147dc4073217/auto_sklearn-0.14.6-py3-none-any.whl
Processing ./.cache/pip/wheels/a7/0b/c3/169e35bcd72f20d0d5e24c37dd2dff8282cc16c06df9762ff5/pynisher-0.6.4-py3-none-any.whl
  Using cached pyrfr-0.8.2-cp38-cp38-manylinux2014_x86_64.whl (3.9 MB)
  Using cached scikit_learn-0.24.2-cp38-cp38-manylinux2010_x86_64.whl (24.9 MB)
  Using cached scipy-1.8.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.6 MB)
  Using cached ConfigSpace-0.4.21-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)
Processing ./.cache/pip/wheels/45/0f/5f/35401965e706ae992e05342841c5ad8b2eb339eb56b0f94d74/smac-1.3.1-py3-none-any.whl
  Using cached smac-1.3.tar.gz (167 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Processing ./.cache/pip/wheels/1c/0d/63/2

  Using cached locket-1.0.0-py2.py3-none-any.whl (4.4 kB)
You should consider upgrading via the '/share/apps/anaconda3/2020.07/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sklearn.datasets
import sklearn.model_selection
import autosklearn.classification
import autosklearn.metrics
import timeit
import pandas as pd

In [2]:
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  return (X_train, X_test, y_train, y_test)

In [3]:
X_train, X_test, y_train, y_test = dataSetup("adult")

/home/av3040/.local/lib/python3.8/site-packages/sklearn/datasets/_openml.py:404: UserWarning: Multiple active versions of the dataset matching the name adult exist. Versions may be fundamentally different, returning version 1.
  warn("Multiple active versions of the dataset matching the name"


In [4]:
X_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country
27859,1,2,19331,3,3,4,10,0,0,0,2,0,0,0
5654,3,2,3413,0,0,0,10,0,0,0,1,0,0,0
3779,4,2,3557,1,1,2,7,4,0,0,1,0,0,0
10522,4,1,8898,0,0,0,3,0,0,0,1,0,1,0
22461,3,2,16419,5,5,0,3,3,0,1,1,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,2,2,9432,0,0,1,1,1,2,0,1,3,2,-1
44732,3,2,26876,1,1,0,9,3,0,1,1,0,0,0
38158,3,2,24238,1,1,2,0,0,0,1,1,0,0,0
860,3,2,852,2,2,0,0,3,0,1,1,0,1,0


In [5]:
start = timeit.default_timer()
estimator_askl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=3600,seed=42,resampling_strategy='cv',metric=autosklearn.metrics.roc_auc,)
estimator_askl.fit(X_train, y_train, dataset_name='adult')
stop = timeit.default_timer()
print(stop - start)
prediction = estimator_askl.predict(X_test)

3597.413639821112


In [6]:
(prediction == y_test).sum()/len(prediction)

0.8572598476783229

In [7]:
print(estimator_askl.sprint_statistics())

auto-sklearn results:
  Dataset name: adult
  Metric: roc_auc
  Best validation score: 0.912072
  Number of target algorithm runs: 38
  Number of successful target algorithm runs: 24
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 6



In [8]:
from pprint import pprint
pprint(estimator_askl.show_models(), indent=4)

{   7: {   'cost': 0.0899666459999006,
           'ensemble_weight': 0.08,
           'estimators': [   {   'balancing': Balancing(random_state=42, strategy='weighting'),
                                 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x14a3e60916d0>,
                                 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x14a3e60eb1c0>,
                                 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x14a3e6086a30>,
                                 'sklearn_classifier': HistGradientBoostingClassifier(early_stopping=True,
                               l2_regularization=7.154474114149067e-10,
                               learning_rate=0.03055499033114984, max_iter=512,
                               max_leaf_nodes=32, n_iter_no_change=5,
                               random_st

                                  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x14a3d5df3e80>,
                                  'sklearn_classifier': HistGradientBoostingClassifier(early_stopping=True,
                               l2_regularization=7.097586934623314e-09,
                               learning_rate=0.16111006880382014, max_iter=64,
                               max_leaf_nodes=8, min_samples_leaf=25,
                               n_iter_no_change=18, random_state=42,
                               validation_fraction=0.05524768284298733,
                               warm_start=True)},
                              {   'balancing': Balancing(random_state=42),
                                  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x14a3d56436a0>,
                                  'data_preprocessor': <autosklearn.pipeline.components.data_prepro

In [9]:
for run_key in estimator_askl.automl_.runhistory_.data:
    print('#########')
    print(run_key)
    print(estimator_askl.automl_.runhistory_.data[run_key])

#########
RunKey(config_id=1, instance_id='{"task_id": "adult"}', seed=0, budget=0.0)
RunValue(cost=1.0, time=54.75899147987366, status=<StatusType.MEMOUT: 5>, starttime=1651431737.9905252, endtime=1651431793.855499, additional_info={'error': 'Memout (used more than 3072 MB).', 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=2, instance_id='{"task_id": "adult"}', seed=0, budget=0.0)
RunValue(cost=1.0, time=59.97691774368286, status=<StatusType.MEMOUT: 5>, starttime=1651431793.9407413, endtime=1651431855.0279663, additional_info={'error': 'Memout (used more than 3072 MB).', 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=3, instance_id='{"task_id": "adult"}', seed=0, budget=0.0)
RunValue(cost=0.15060777477951834, time=20.585455417633057, status=<StatusType.SUCCESS: 1>, starttime=1651431855.0845037, endtime=1651431875.706117, additional_info={'duration': 20.13684892654419, 'num_run': 4, 'train_loss': 0.07042387620581354, 'configuration_origin':